In [1]:
import cfbd
import pandas as pd
import json

CFBD_API_KEY = ""

# Get API Key from secrets.json
with open('secrets.json') as f:
    secrets = json.load(f)
    CFBD_API_KEY = secrets["CFBD_API_KEY"]
    
# Configure API key authorization
configuration = cfbd.Configuration(
    access_token=CFBD_API_KEY,  host="https://api.collegefootballdata.com")

print("Using CFBD API key:", CFBD_API_KEY)

Using CFBD API key: q7ytOkNtIxdIVFT0kdFj720IEJ8valixpxI1cTtpxjpkGDPLWftm9CWj5C4Arn3v


In [2]:
season = 2025
week = 6

In [3]:
with cfbd.ApiClient(configuration) as api_client:
	betting_api = cfbd.BettingApi(api_client)

	try:
		lines = betting_api.get_lines(year=season, week=week, provider='DraftKings')
		lines_df = pd.DataFrame([line.to_dict() for line in lines])

		# Extract the first object from the 'lines' list in each row
		lines_df['lines_first'] = lines_df['lines'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)

		# Now extract properties from the first object
		lines_df['spread'] = lines_df['lines_first'].apply(lambda x: x.get('spread') if isinstance(x, dict) and 'spread' in x else None)
		lines_df['formatted_spread'] = lines_df['lines_first'].apply(lambda x: x.get('formattedSpread') if isinstance(x, dict) and 'formattedSpread' in x else None)
		lines_df['overUnder'] = lines_df['lines_first'].apply(lambda x: x.get('overUnder') if isinstance(x, dict) and 'overUnder' in x else None)
		lines_df['homeMoneyline'] = lines_df['lines_first'].apply(lambda x: x.get('homeMoneyline') if isinstance(x, dict) and 'homeMoneyline' in x else None)
		lines_df['awayMoneyline'] = lines_df['lines_first'].apply(lambda x: x.get('awayMoneyline') if isinstance(x, dict) and 'awayMoneyline' in x else None)

		lines_df = lines_df.drop(columns=['lines', 'lines_first', 'id', 'seasonType', 'homeConference', 'awayConference', 'homeClassification', 'awayClassification'])

		lines_df.to_csv(f'cfbd_lines_{season}_week{week}.csv', index=False)
	except cfbd.ApiException as e:
		print("Exception when calling BettingApi->get_lines: %s\n" % e)